In [0]:
%sql
DROP SCHEMA IF EXISTS cpg_industry.gold CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS cpg_industry.gold;

In [0]:
%sql
WITH sales_data AS (
    SELECT 
        oi.product_id,
        SUM(oi.total_price) AS total_sales_value,
        SUM(oi.quantity) AS total_sold_qty,
        COUNT(*) AS total_order_items
    FROM cpg_industry.bronze.cpg_consumer_order_items oi
    JOIN cpg_industry.bronze.cpg_consumer_order o 
        ON oi.order_id = o.order_id
    GROUP BY oi.product_id
),
inventory_data AS (
    SELECT 
        inv.product_id,
        inv.quantity_on_hand,
        p.unit_price,
        p.retail_price
    FROM cpg_industry.bronze.cpg_inventory inv
    JOIN cpg_industry.bronze.cpg_product p 
        ON inv.product_id = p.product_id
),
weekly_sales AS (
    SELECT 
        oi.product_id,
        SUM(oi.quantity) / COUNT(DISTINCT weekofyear(o.order_date)) AS qty_per_week
    FROM cpg_industry.bronze.cpg_consumer_order_items oi
    JOIN cpg_industry.bronze.cpg_consumer_order o 
        ON oi.order_id = o.order_id
    GROUP BY oi.product_id
),
daily_sales AS (
    SELECT 
        oi.product_id,
        SUM(oi.quantity) / COUNT(DISTINCT o.order_date) AS qty_per_day
    FROM cpg_industry.bronze.cpg_consumer_order_items oi
    JOIN cpg_industry.bronze.cpg_consumer_order o 
        ON oi.order_id = o.order_id
    GROUP BY oi.product_id
)
SELECT
    -- 1. Stock to Sales Ratio
    ROUND(SUM(inv.quantity_on_hand * inv.retail_price) / NULLIF(SUM(sd.total_sales_value), 0), 2) AS stock_to_sales_ratio,

    -- 2. Inventory Turnover Rate
    ROUND(SUM(sd.total_sales_value) / NULLIF(SUM(inv.quantity_on_hand * inv.unit_price), 0), 2) AS inventory_turnover_rate,

    -- 3. Weeks On-Hand
    ROUND(SUM(inv.quantity_on_hand) / NULLIF(AVG(ws.qty_per_week), 0), 2) AS weeks_on_hand,

    -- 4. Backorder Rate (%)
    ROUND(SUM(CASE WHEN inv.quantity_on_hand < sd.total_sold_qty THEN 1 ELSE 0 END) 
          / NULLIF(SUM(sd.total_order_items), 0) * 100, 2) AS backorder_rate_percentage,

    -- 5. Days Sales in Inventory
    ROUND(SUM(inv.quantity_on_hand) / NULLIF(AVG(ds.qty_per_day), 0), 2) AS days_sales_in_inventory,

    -- 6. Perfect Order Rate (%)
    (
        SELECT ROUND(SUM(CASE WHEN order_status = 'Delivered' THEN 1 ELSE 0 END) / COUNT(*) * 100, 2)
        FROM cpg_industry.bronze.cpg_consumer_order
    ) AS perfect_order_rate_percentage,

    -- 7. Sell-Through Rate (%)
    ROUND(SUM(sd.total_sold_qty) / NULLIF(SUM(sd.total_sold_qty) + SUM(inv.quantity_on_hand), 0) * 100, 2) AS sell_through_rate_percentage

FROM inventory_data inv
LEFT JOIN sales_data sd  ON inv.product_id = sd.product_id
LEFT JOIN weekly_sales ws ON inv.product_id = ws.product_id
LEFT JOIN daily_sales ds  ON inv.product_id = ds.product_id;


### Out_of_Stock

In [0]:
%sql
CREATE OR REPLACE  TABLE cpg_industry.gold.out_of_stock AS
SELECT
    -- KPI calculation
    COUNT(DISTINCT CASE 
        WHEN quantity_on_hand = 0 OR inventory_status = 'Low Stock' 
        THEN product_id 
    END) AS stockout_count,
    
    COUNT(DISTINCT product_id) AS total_skus,
    
    ROUND(
        (COUNT(DISTINCT CASE 
            WHEN quantity_on_hand = 0 OR inventory_status = 'Low Stock' 
            THEN product_id 
        END) * 100.0) / COUNT(DISTINCT product_id),
        2
    ) AS out_of_stock_rate_percentage
FROM cpg_industry.silver.cpg_inventory
WHERE location_is_active = true;


In [0]:
%sql
-- CREATE OR REPLACE TABLE cpg_industry.gold.gold_clv_consumer AS
SELECT
    c.consumer_id,
    c.registration_date,
    MIN(o.order_date) AS first_order_date,
    MAX(o.order_date) AS last_order_date,
    DATEDIFF(MAX(o.order_date), MIN(o.order_date)) AS lifespan_days,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(i.net_amount) AS total_revenue,
    ROUND(SUM(i.net_amount) / COUNT(DISTINCT o.order_id), 2) AS avg_purchase_value,
    ROUND(COUNT(DISTINCT o.order_id) / NULLIF(DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 0), 2) AS purchase_frequency,
    ROUND(
        (SUM(i.net_amount) / COUNT(DISTINCT o.order_id)) *
        (COUNT(DISTINCT o.order_id) / NULLIF(DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 0)) *
        DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 2
    ) AS clv
FROM cpg_industry.silver.cpg_consumer c
JOIN cpg_industry.silver.cpg_consumer_order o ON c.consumer_id = o.consumer_id
JOIN cpg_industry.silver.cpg_consumer_invoice i ON o.order_id = i.order_id
WHERE c.is_active = TRUE
GROUP BY c.consumer_id, c.registration_date;


In [0]:
%sql
-- B2C Out-of-Stock Rate
-- CREATE OR REPLACE TABLE cpg_industry.gold.gold_out_of_stock_rate_b2c AS
SELECT
    CURRENT_DATE() AS report_date,
    COUNT(DISTINCT CASE 
                      WHEN inv.quantity_on_hand = 0 
                           OR LOWER(inv.inventory_status) = 'low stock' 
                      THEN inv.product_id 
                   END) AS stockout_count,
    COUNT(DISTINCT inv.product_id) AS total_skus,
    ROUND(
        (COUNT(DISTINCT CASE 
                           WHEN inv.quantity_on_hand = 0 
                                OR LOWER(inv.inventory_status) = 'low stock' 
                           THEN inv.product_id 
                        END) * 100.0) 
        / COUNT(DISTINCT inv.product_id), 2
    ) AS out_of_stock_rate
FROM cpg_industry.silver.cpg_inventory AS inv
JOIN cpg_industry.silver.cpg_product AS p 
    ON inv.product_id = p.product_id
WHERE LOWER(p.product_status) = 'active';


### Sales by category

In [0]:
%sql
CREATE OR REPLACE TABLE cpg_industry.gold.sales_by_category AS
SELECT 
    p.category AS product_category,
    ROUND(SUM(oi.total_price), 2) AS total_revenue
FROM cpg_industry.silver.cpg_order_items oi
JOIN cpg_industry.silver.cpg_product p
    ON oi.product_id = p.product_id
JOIN cpg_industry.silver.cpg_consumer_order o
    ON oi.order_id = o.order_id
JOIN  cpg_industry.silver.cpg_consumer_invoice i
    ON o.order_id = i.order_id
WHERE o.order_status = 'Delivered'
  AND i.invoice_status = 'Refunded'
GROUP BY p.category
ORDER BY total_revenue DESC;

SELECT * FROM cpg_industry.gold.sales_by_category;